# Week 4: Text and time

1. Text
    - Dealing with text data
    - Cleaning dirty integer data
    - Textual statistics 
    - Trimming strings
2. Dates and times
    - What does it mean to have dates and times in programming / data?
    - Time deltas
    - Time series
    - Resampling 